1) Pull tweets from twitter over a time period
    - Only use 'Verified' accounts
    - Only use 'relevant' tweets: If they contain at least one of the stock symbols and names (crpyto and names)
2) Label the tweets (Can skip this - do sentiments on tweets)
    - Is the tweet beneficial to the stock or not (rating of -2 to +2)
    - Assign neutrality if between -0.5, +0.5 and positive/negative otherwise
3) Preprocess the tweets
    - Tokenize, etc
    - Replace all numbers with <num> tag
    - Remove some punctuation (-, .)
    - Modify Loughrans lexicon of positive and negative words to be suited for a stock market context

4) Download stock data
    - Closing price, for all the stock symbols and names you need
    - Calculate the relative daily return to create a stationary time-series
    
5) First determine if time series is stationary (KPSS)

https://www.aclweb.org/anthology/W18-3102.pdf#page=7&zoom=100,89,512

In [30]:
import requests
import os
import json

In [80]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAKeALAEAAAAA7pr%2B0Wbti3cRC%2FWaMWouen%2F2cmQ%3DLdwugPjXpcX8TqRrocCtqAAYFwUP8dDqF1jMgm26kk8o2bVvcx'

In [259]:
all_verified_tweets = []
all_tweets = []

In [260]:
def create_url(query):
    query = query
    tweet_fields = "tweet.fields=author_id"
    user_fields = "user.fields=entities"
    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}&{}".format(
        query, user_fields, tweet_fields
    )
    return url

In [261]:
def create_url_verified(id):
    url = f"https://api.twitter.com/2/users?ids={id}&user.fields=verified"
    return url

In [262]:
def get_tweets(query):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    url = create_url(query)
    r = requests.request("GET", url, headers = headers)
    return r

def verify_tweets(r):
    tweets_verified = []
    tweets_normal = []
    for tweet in r.json()['data']:
        url = create_url_verified(tweet['author_id'])
        r = requests.request("GET", url, headers = headers)
        verified = r.json()['data'][0]['verified']
        if verified:
            print(tweet)
            tweets_verified.append(tweet)
        else:
            tweets_normal.append(tweet)
    all_tweets.append(tweets_normal)
    all_verified_tweets.append(tweets_verified)
    return tweets

In [267]:
tweets = get_tweets("bitcoin")
verified = verify_tweets(tweets)
verified

<Response [200]>

In [271]:
all_verified_tweets

[[{'author_id': '2260491445',
   'id': '1366268214754357248',
   'text': 'Almost every decentralized network claims a very high level of decentralized — but is that always the case?\n\n#Crypto #Cryptocurrency #Altcoin #Blockchain #Bitcoin \n\nhttps://t.co/YWWviFAixV'}],
 []]

In [272]:
all_tweets

[[{'author_id': '1260968439931420677',
   'id': '1366268218139099136',
   'text': 'I am sending you 1π! Pi is a new digital currency developed by Stanford PhDs with over 10 million members worldwide. To claim your Pi follow this link and use my username (borgonovorob) as your invitation code. #bitcoin; https://t.co/2ySFWchgnV'},
  {'author_id': '1345727073466392579',
   'id': '1366268217782591491',
   'text': 'The MACD supports a bullish outlook suggesting that bulls still have a chance to take control of the trend.#ForexShakeel #crytpo #bitcoin #btc #news #cardano #ada #xrp #doge #Algorand #polkadot #Dogecoins #RippleNet #Ripple #ETH #Tezos #TezosAnalysis #XLMUP #cryptocurrency #Link https://t.co/o8pFi2y69n'},
  {'author_id': '1221366000752324609',
   'id': '1366268216092233730',
   'text': 'RT @News24: Creditors of the disgraced trading scheme will reconvene in court on Monday to hear whether it should be permanently liquidated…'},
  {'author_id': '1188089088085286914',
   'id': '136